# 1 Converting strings to DateTime

In [ ]:
import pandas as pd

#covert list of strings to datetime
df1 = pd.DataFrame(
     {
      "DateTime": pd.to_datetime(["2021-11-13 13:45:33", '2021-11-14 14:13:25',"2021-11-15 14:39:25", "2021-11-16 15:16:43", "2021-11-17 16:51:19"]),
     }
)
df1


,DateTime
0,2021-11-13 13:45:33
1,2021-11-14 14:13:25
2,2021-11-15 14:39:25
3,2021-11-16 15:16:43
4,2021-11-17 16:51:19


In [ ]:
#import pandas as pd
import datetime as dt

#split a datetime column into individual date and time columns
df1['Dates'] = pd.to_datetime(df1['DateTime']).dt.date
df1['Time'] = pd.to_datetime(df1['DateTime']).dt.time
df1

,DateTime,Dates,Time
0,2021-11-13 13:45:33,2021-11-13,13:45:33
1,2021-11-14 14:13:25,2021-11-14,14:13:25
2,2021-11-15 14:39:25,2021-11-15,14:39:25
3,2021-11-16 15:16:43,2021-11-16,15:16:43
4,2021-11-17 16:51:19,2021-11-17,16:51:19


In [ ]:
#join date and time column into a single datetime column 
df1['DateTime_join'] = pd.DataFrame(pd.to_datetime(df1['Dates'].astype(str) + ' ' + df1['Time'].astype(str)), columns=['Datetime_join']) 
df1

,DateTime,Dates,Time,DateTime_join
0,2021-11-13 13:45:33,2021-11-13,13:45:33,2021-11-13 13:45:33
1,2021-11-14 14:13:25,2021-11-14,14:13:25,2021-11-14 14:13:25
2,2021-11-15 14:39:25,2021-11-15,14:39:25,2021-11-15 14:39:25
3,2021-11-16 15:16:43,2021-11-16,15:16:43,2021-11-16 15:16:43
4,2021-11-17 16:51:19,2021-11-17,16:51:19,2021-11-17 16:51:19


#2. Working with Timezone

In [ ]:
import pandas as pd
from datetime import datetime
import pytz, tzlocal

#convert a aware datetime to naive datetime
dt_aware = pd.to_datetime('2021-11-26T11:04:30+02:00')
germany = pytz.timezone("Europe/Berlin")

dt_naive = dt_aware.astimezone(germany).replace(tzinfo=None)
print(dt_naive)

2021-11-26 10:04:30


In [ ]:
#convert a naive datetime to an aware datetime
dt_naive = pd.to_datetime('2021-11-27T12:24:30')
japan = pytz.timezone("Asia/Tokyo")
dt_aware = dt_naive.tz_localize(japan)
print(dt_aware)

2021-11-27 12:24:30+09:00


In [ ]:
import tzlocal

#convert a aware datetime to local naive timezone (in Google Colab the local timezone is utc )
dt_aware = pd.to_datetime('2021-11-28T11:04:30+02:00')
local_timezone = tzlocal.get_localzone()
dt_local = dt_aware.astimezone(local_timezone).replace(tzinfo=None)
print(dt_local)

2021-11-28 09:04:30


#3. Conditionally compare two DateTime(s)

In [37]:
import pandas as pd
import datetime

df1 = pd.DataFrame(
    pd.to_datetime(["13:45:33", '14:17:25',"14:13:25", "15:16:43", "16:51:19"]),
    columns=["MsgTime"],
)

df1

,MsgTime
0,2021-12-12 13:45:33
1,2021-12-12 14:17:25
2,2021-12-12 14:13:25
3,2021-12-12 15:16:43
4,2021-12-12 16:51:19


In [38]:
df2 = pd.DataFrame(
    {
        "MsgTime1": pd.to_datetime(["13:40:33", "14:16:43", "16:49:11"]),
        "MsgTime2": pd.to_datetime(["13:50:13", "14:26:43", "16:59:02"]),
        "Temperature":[21,22,32],
        "Humidity":[45,56,40],
    }
)
df2

,MsgTime1,MsgTime2,Temperature,Humidity
0,2021-12-12 13:40:33,2021-12-12 13:50:13,21,45
1,2021-12-12 14:16:43,2021-12-12 14:26:43,22,56
2,2021-12-12 16:49:11,2021-12-12 16:59:02,32,40


In [40]:
df2.index = df2[["MsgTime1", "MsgTime2"]].apply(lambda x: pd.Interval(*pd.to_datetime(x)), axis=1)
df2

,MsgTime1,MsgTime2,Temperature,Humidity
"(2021-12-12 13:40:33, 2021-12-12 13:50:13]",2021-12-12 13:40:33,2021-12-12 13:50:13,21,45
"(2021-12-12 14:16:43, 2021-12-12 14:26:43]",2021-12-12 14:16:43,2021-12-12 14:26:43,22,56
"(2021-12-12 16:49:11, 2021-12-12 16:59:02]",2021-12-12 16:49:11,2021-12-12 16:59:02,32,40


In [41]:
for j in ["Temperature", "Humidity"]:
    df1[j] = df1["MsgTime"].map(df2[j])
df1

,MsgTime,Temperature,Humidity
0,2021-12-12 13:45:33,21.0,45.0
1,2021-12-12 14:17:25,22.0,56.0
2,2021-12-12 14:13:25,NaN,NaN
3,2021-12-12 15:16:43,NaN,NaN
4,2021-12-12 16:51:19,32.0,40.0
